In [1]:
!pip install llama-index-embeddings-azure-openai
!pip install llama-index-llms-azure-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 162.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.6/477.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 4.8 MB/s eta 0:00:00


In [1]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
api_key = "0140412acf0e4f69a00b0c63c454a2e8"
azure_endpoint = "https://thangtd.openai.azure.com/"
api_version = "2023-03-15-preview"
llm = AzureOpenAI(
    model="GPT35",
    deployment_name="gpt",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ADA002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [5]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import FlatReader, HTMLTagReader
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader(input_dir="data_v2/cleaned_data")

docs = reader.load_data()

In [6]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.extractors import TitleExtractor

import qdrant_client


client = qdrant_client.QdrantClient(location=":memory:")
vector_store = QdrantVectorStore(client=client, collection_name="test_store")

pipeline = IngestionPipeline(
    documents=docs,
    transformations=[
        HTMLNodeParser.from_defaults(include_metadata=True, include_prev_next_rel=True),
        SemanticSplitterNodeParser(
            buffer_size=4, embed_model=embed_model, include_metadata=True, include_prev_next_rel=True
        ),
        embed_model,
    ],
    vector_store=vector_store,
)

In [7]:
results = pipeline.run()

INFO:httpx:HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [12]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(results)

ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [3]:
import os
from llama_index.core import Settings


Settings.embed_model = embed_model

PERSIST_DIR = "database"
if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader(input_dir="data_v2/cleaned_data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir="database/")